In [ ]:
from run_on_azure_batch import *

In [ ]:
image_name = IMAGE_NAME
image_version = IMAGE_VERSION

In [ ]:
image_name

In [ ]:
credentials = batch_auth.SharedKeyCredentials(
    config.BATCH_ACCOUNT_NAME, config.BATCH_ACCOUNT_KEY
)

In [ ]:
batch_client = batch.BatchServiceClient(
    credentials, base_url=config.BATCH_ACCOUNT_URL
)

In [ ]:
job_id = "Job-" + "{:%Y-%m-%d-%H-%M-%S}".format(datetime.datetime.now())
pool_id = "titanic"

In [ ]:
container_registry = batch.models.ContainerRegistry(
            registry_server=config.ACR_LOGINSERVER,
            user_name=config.ACR_USERNAME,
            password=config.ACR_PASSWORD,
            )

create_pool(
    batch_service_client=batch_client,
    pool_id=pool_id,
    container_registry=container_registry,
    image_name=image_name,
    pool_node_count=3,
    pool_vm_size="Standard_E2s_v3",
    skip_if_exists=True,
)

In [ ]:
    try:
        # Create the job that will run the tasks.
        create_job(batch_client, job_id, pool_id)

        tasks = []
        
        for pclass in [1]:
            command = f"python /src/titanic_analytics.py --file wasbs://titanic@sparkonbatch.blob.core.windows.net/train.csv --pclass {pclass} --output wasbs://titanic@sparkonbatch.blob.core.windows.net/output_{pclass}.parquet"
            task = add_task(
                batch_service_client=batch_client,
                image_name=image_name,
                image_version=image_version,
                job_id=job_id,
                command=command,
                name=f"pclass{pclass}",
            )
            tasks.append(task)

        # Pause execution until tasks reach Completed state.
        wait_for_tasks_to_complete(batch_client, job_id, datetime.timedelta(hours=2))

        print("  Success! All tasks reached the 'Completed' state within the specified timeout period.")

    except batchmodels.BatchErrorException as err:
        print(err)
        raise

In [ ]:
# batch_client.job.delete(job_id)

In [ ]:
# batch_client.pool.delete("titanic")